In [1]:
!pip install numpy==1.22.4
!pip install pandas==1.3.4
!pip install seaborn==0.11.2
!pip install imbalanced-learn==0.9.1

ERROR: Could not find a version that satisfies the requirement imblearn==0.9.1 (from versions: 0.0)
ERROR: No matching distribution found for imblearn==0.9.1


In [2]:
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

df = pd.read_csv('train_set_github.csv')
df

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,Label
0,ENST00000003100,ENSG00000001630,1021,ATGAC,4.764186,89.360465,0.006822,4.400,89.70,0.005980,0
1,ENST00000003100,ENSG00000001630,1022,TGACA,9.278605,115.000000,0.006003,9.190,116.00,0.004670,0
2,ENST00000003100,ENSG00000001630,1023,GACAT,2.570698,80.306977,0.007468,2.300,80.10,0.006310,0
3,ENST00000003100,ENSG00000001630,1071,TTGAC,3.195217,106.693478,0.007351,3.155,107.00,0.006605,0
4,ENST00000003100,ENSG00000001630,1072,TGACT,6.813261,123.130435,0.009585,6.940,123.00,0.008845,0
...,...,...,...,...,...,...,...,...,...,...,...
111031,ENST00000641784,ENSG00000284707,3243,GAACA,2.998125,97.346875,0.007648,2.675,97.60,0.006580,0
111032,ENST00000641784,ENSG00000284707,3244,AACAA,2.203750,88.439063,0.005190,2.130,88.70,0.004660,0
111033,ENST00000641784,ENSG00000284707,3265,CTAAC,1.874516,94.209677,0.005972,1.760,94.80,0.005065,0
111034,ENST00000641784,ENSG00000284707,3266,TAACT,2.194032,99.730645,0.006831,2.170,99.75,0.005785,0


In [3]:
# Obtain all unique gene names in the dataframe
gene_names = []
for i in df['Gene Name']:
    if i not in gene_names:
        gene_names.append(i)

# Split dataset into training and validation data (70/30 ratio) based on the Gene name
random.seed(4262)
training_genes = random.sample(gene_names, int(0.7 * len(gene_names)) ) # sample 70% of genes to be used in training
validation_genes = list(set(gene_names) - set(training_genes)) # remainder of genes that are not sampled will be used in validation
training_data = df[df['Gene Name'].isin(training_genes)]
validation_data = df[df['Gene Name'].isin(validation_genes)]

training_y = training_data['Label'].reset_index(drop=True)
training_X = training_data.drop(['Label'], axis=1).reset_index(drop=True)
validation_y = validation_data['Label'].reset_index(drop=True)
validation_X = validation_data.drop(['Label'], axis=1).reset_index(drop=True)

# One hot encoding for bases column of train and test set
training_X_dummies = pd.get_dummies(training_X['Bases'], drop_first=True)
training_X = pd.concat([training_X, training_X_dummies],axis=1)
validation_X_dummies = pd.get_dummies(validation_X['Bases'], drop_first=True)
validation_X = pd.concat([validation_X, validation_X_dummies],axis=1)

training_X

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,...,TAACC,TAACT,TAGAC,TGAAC,TGACA,TGACC,TGACT,TGGAC,TTAAC,TTGAC
0,ENST00000005257,ENSG00000006451,469,CTGAC,4.900645,111.441935,0.008884,3.840,112.00,0.006430,...,0,0,0,0,0,0,0,0,0,0
1,ENST00000005257,ENSG00000006451,470,TGACT,9.120968,121.032258,0.010946,9.260,121.00,0.008800,...,0,0,0,0,0,0,1,0,0,0
2,ENST00000005257,ENSG00000006451,471,GACTC,2.513839,90.670968,0.012280,2.390,90.80,0.009630,...,0,0,0,0,0,0,0,0,0,0
3,ENST00000005257,ENSG00000006451,503,AGGAC,6.078824,116.970588,0.012514,6.040,117.00,0.012050,...,0,0,0,0,0,0,0,0,0,0
4,ENST00000005257,ENSG00000006451,504,GGACT,4.955000,126.029412,0.011593,4.920,126.00,0.010300,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,ENST00000641784,ENSG00000284707,3243,GAACA,2.998125,97.346875,0.007648,2.675,97.60,0.006580,...,0,0,0,0,0,0,0,0,0,0
77117,ENST00000641784,ENSG00000284707,3244,AACAA,2.203750,88.439063,0.005190,2.130,88.70,0.004660,...,0,0,0,0,0,0,0,0,0,0
77118,ENST00000641784,ENSG00000284707,3265,CTAAC,1.874516,94.209677,0.005972,1.760,94.80,0.005065,...,0,0,0,0,0,0,0,0,0,0
77119,ENST00000641784,ENSG00000284707,3266,TAACT,2.194032,99.730645,0.006831,2.170,99.75,0.005785,...,0,1,0,0,0,0,0,0,0,0


In [4]:
training_data

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,Label
120,ENST00000005257,ENSG00000006451,469,CTGAC,4.900645,111.441935,0.008884,3.840,112.00,0.006430,0
121,ENST00000005257,ENSG00000006451,470,TGACT,9.120968,121.032258,0.010946,9.260,121.00,0.008800,0
122,ENST00000005257,ENSG00000006451,471,GACTC,2.513839,90.670968,0.012280,2.390,90.80,0.009630,0
123,ENST00000005257,ENSG00000006451,503,AGGAC,6.078824,116.970588,0.012514,6.040,117.00,0.012050,0
124,ENST00000005257,ENSG00000006451,504,GGACT,4.955000,126.029412,0.011593,4.920,126.00,0.010300,0
...,...,...,...,...,...,...,...,...,...,...,...
111031,ENST00000641784,ENSG00000284707,3243,GAACA,2.998125,97.346875,0.007648,2.675,97.60,0.006580,0
111032,ENST00000641784,ENSG00000284707,3244,AACAA,2.203750,88.439063,0.005190,2.130,88.70,0.004660,0
111033,ENST00000641784,ENSG00000284707,3265,CTAAC,1.874516,94.209677,0.005972,1.760,94.80,0.005065,0
111034,ENST00000641784,ENSG00000284707,3266,TAACT,2.194032,99.730645,0.006831,2.170,99.75,0.005785,0


In [5]:
validation_data

,Transcript Name,Gene Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,Label
0,ENST00000003100,ENSG00000001630,1021,ATGAC,4.764186,89.360465,0.006822,4.400,89.7,0.005980,0
1,ENST00000003100,ENSG00000001630,1022,TGACA,9.278605,115.000000,0.006003,9.190,116.0,0.004670,0
2,ENST00000003100,ENSG00000001630,1023,GACAT,2.570698,80.306977,0.007468,2.300,80.1,0.006310,0
3,ENST00000003100,ENSG00000001630,1071,TTGAC,3.195217,106.693478,0.007351,3.155,107.0,0.006605,0
4,ENST00000003100,ENSG00000001630,1072,TGACT,6.813261,123.130435,0.009585,6.940,123.0,0.008845,0
...,...,...,...,...,...,...,...,...,...,...,...
110467,ENST00000634753,ENSG00000282947,2333,GAACT,3.808538,100.923077,0.009656,3.330,100.0,0.007300,0
110468,ENST00000634753,ENSG00000282947,2334,AACTG,2.351154,93.513077,0.013745,2.255,93.6,0.012750,0
110469,ENST00000634753,ENSG00000282947,2352,TTAAC,1.922843,90.948819,0.006231,1.720,91.6,0.005310,0
110470,ENST00000634753,ENSG00000282947,2353,TAACC,2.673543,92.984252,0.006830,2.560,93.2,0.005660,0


### Feature engineering functions

In [6]:
# Obtain the counts of the individual bases and use them as features
def count_bases(bases):
    a,t,c,g=0,0,0,0
    for i in bases:
        if i == 'A':
            a+=1
        elif i == 'T':
            t+=1
        elif i == 'C':
            c+=1
        else:
            g+=1
    return a,t,c,g

In [7]:
# Relative positions
relative_positions = [1,2,3] * int(len(training_X)/3)
relative_positions_df = pd.DataFrame(relative_positions, columns=['Relative Position'])
training_X = pd.concat([training_X, relative_positions_df],axis=1)

training_X['Count_A'], training_X['Count_T'], training_X['Count_C'], training_X['Count_G'] = zip(*training_X['Bases'].apply(count_bases))
training_X = training_X.drop(['Bases'],axis=1)

training_X

,Transcript Name,Gene Name,Position,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,AAACA,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000005257,ENSG00000006451,469,4.900645,111.441935,0.008884,3.840,112.00,0.006430,0,...,0,0,0,0,0,1,1,1,2,1
1,ENST00000005257,ENSG00000006451,470,9.120968,121.032258,0.010946,9.260,121.00,0.008800,0,...,0,1,0,0,0,2,1,2,1,1
2,ENST00000005257,ENSG00000006451,471,2.513839,90.670968,0.012280,2.390,90.80,0.009630,0,...,0,0,0,0,0,3,1,1,2,1
3,ENST00000005257,ENSG00000006451,503,6.078824,116.970588,0.012514,6.040,117.00,0.012050,0,...,0,0,0,0,0,1,2,0,1,2
4,ENST00000005257,ENSG00000006451,504,4.955000,126.029412,0.011593,4.920,126.00,0.010300,0,...,0,0,0,0,0,2,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,ENST00000641784,ENSG00000284707,3243,2.998125,97.346875,0.007648,2.675,97.60,0.006580,0,...,0,0,0,0,0,2,3,0,1,1
77117,ENST00000641784,ENSG00000284707,3244,2.203750,88.439063,0.005190,2.130,88.70,0.004660,0,...,0,0,0,0,0,3,4,0,1,0
77118,ENST00000641784,ENSG00000284707,3265,1.874516,94.209677,0.005972,1.760,94.80,0.005065,0,...,0,0,0,0,0,1,2,1,2,0
77119,ENST00000641784,ENSG00000284707,3266,2.194032,99.730645,0.006831,2.170,99.75,0.005785,0,...,0,0,0,0,0,2,2,2,1,0


In [8]:
# Relative positions
relative_positions = [1,2,3] * int(len(validation_X)/3)
relative_positions_df = pd.DataFrame(relative_positions, columns=['Relative Position'])
validation_X = pd.concat([validation_X, relative_positions_df],axis=1)

validation_X['Count_A'], validation_X['Count_T'], validation_X['Count_C'], validation_X['Count_G'] = zip(*validation_X['Bases'].apply(count_bases)) 
validation_X = validation_X.drop(['Bases'],axis=1)

validation_X

,Transcript Name,Gene Name,Position,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,AAACA,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000003100,ENSG00000001630,1021,4.764186,89.360465,0.006822,4.400,89.7,0.005980,0,...,0,0,0,0,0,1,2,1,1,1
1,ENST00000003100,ENSG00000001630,1022,9.278605,115.000000,0.006003,9.190,116.0,0.004670,0,...,0,0,0,0,0,2,2,1,1,1
2,ENST00000003100,ENSG00000001630,1023,2.570698,80.306977,0.007468,2.300,80.1,0.006310,0,...,0,0,0,0,0,3,2,1,1,1
3,ENST00000003100,ENSG00000001630,1071,3.195217,106.693478,0.007351,3.155,107.0,0.006605,0,...,0,0,0,0,1,1,1,2,1,1
4,ENST00000003100,ENSG00000001630,1072,6.813261,123.130435,0.009585,6.940,123.0,0.008845,0,...,0,1,0,0,0,2,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33910,ENST00000634753,ENSG00000282947,2333,3.808538,100.923077,0.009656,3.330,100.0,0.007300,0,...,0,0,0,0,0,2,2,1,1,1
33911,ENST00000634753,ENSG00000282947,2334,2.351154,93.513077,0.013745,2.255,93.6,0.012750,0,...,0,0,0,0,0,3,2,1,1,1
33912,ENST00000634753,ENSG00000282947,2352,1.922843,90.948819,0.006231,1.720,91.6,0.005310,0,...,0,0,0,1,0,1,2,2,1,0
33913,ENST00000634753,ENSG00000282947,2353,2.673543,92.984252,0.006830,2.560,93.2,0.005660,0,...,0,0,0,0,0,2,2,1,2,0


## Feature Selection

### Feature selection using Pearson Correlation

In [9]:
df_cor = training_X.drop(['Transcript Name', 'Gene Name'], axis=1)
cor_matrix = df_cor.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)] # Drop features if correlation is > 0.75
df_cor = df_cor.drop(df_cor[to_drop], axis=1)
df_cor

C:\Users\jingh\AppData\Local\Temp/ipykernel_15724/1420394356.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


,Position,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,AACAG,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,469,4.900645,111.441935,0.008884,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,2,1
1,470,9.120968,121.032258,0.010946,0,0,0,0,0,0,...,0,1,0,0,0,2,1,2,1,1
2,471,2.513839,90.670968,0.012280,0,0,0,0,0,0,...,0,0,0,0,0,3,1,1,2,1
3,503,6.078824,116.970588,0.012514,0,0,0,0,0,0,...,0,0,0,0,0,1,2,0,1,2
4,504,4.955000,126.029412,0.011593,0,0,0,0,0,0,...,0,0,0,0,0,2,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,3243,2.998125,97.346875,0.007648,0,0,0,0,0,0,...,0,0,0,0,0,2,3,0,1,1
77117,3244,2.203750,88.439063,0.005190,0,0,0,1,0,0,...,0,0,0,0,0,3,4,0,1,0
77118,3265,1.874516,94.209677,0.005972,0,0,0,0,0,0,...,0,0,0,0,0,1,2,1,2,0
77119,3266,2.194032,99.730645,0.006831,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,1,0


In [10]:
print(to_drop)

['Median SD', 'Median_Mean', 'Median Dwelling Time']


In [11]:
# Drop Position variable if not dealing with relative positions 
training_X = training_X.drop(training_X[to_drop], axis=1).drop(['Position'],axis=1)
validation_X = validation_X.drop(validation_X[to_drop], axis=1).drop(['Position'],axis=1)
training_X

,Transcript Name,Gene Name,Mean SD,Mean_Mean,Mean Dwelling Time,AAACA,AAACC,AAACT,AACAA,AACAC,...,TGACC,TGACT,TGGAC,TTAAC,TTGAC,Relative Position,Count_A,Count_T,Count_C,Count_G
0,ENST00000005257,ENSG00000006451,4.900645,111.441935,0.008884,0,0,0,0,0,...,0,0,0,0,0,1,1,1,2,1
1,ENST00000005257,ENSG00000006451,9.120968,121.032258,0.010946,0,0,0,0,0,...,0,1,0,0,0,2,1,2,1,1
2,ENST00000005257,ENSG00000006451,2.513839,90.670968,0.012280,0,0,0,0,0,...,0,0,0,0,0,3,1,1,2,1
3,ENST00000005257,ENSG00000006451,6.078824,116.970588,0.012514,0,0,0,0,0,...,0,0,0,0,0,1,2,0,1,2
4,ENST00000005257,ENSG00000006451,4.955000,126.029412,0.011593,0,0,0,0,0,...,0,0,0,0,0,2,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,ENST00000641784,ENSG00000284707,2.998125,97.346875,0.007648,0,0,0,0,0,...,0,0,0,0,0,2,3,0,1,1
77117,ENST00000641784,ENSG00000284707,2.203750,88.439063,0.005190,0,0,0,1,0,...,0,0,0,0,0,3,4,0,1,0
77118,ENST00000641784,ENSG00000284707,1.874516,94.209677,0.005972,0,0,0,0,0,...,0,0,0,0,0,1,2,1,2,0
77119,ENST00000641784,ENSG00000284707,2.194032,99.730645,0.006831,0,0,0,0,0,...,0,0,0,0,0,2,2,2,1,0


In [12]:
training_X.columns

Index(['Transcript Name', 'Gene Name', 'Mean SD', 'Mean_Mean',
       'Mean Dwelling Time', 'AAACA', 'AAACC', 'AAACT', 'AACAA', 'AACAC',
       'AACAG', 'AACAT', 'AACCA', 'AACCC', 'AACCG', 'AACCT', 'AACTA', 'AACTC',
       'AACTG', 'AACTT', 'AAGAC', 'AGAAC', 'AGACA', 'AGACC', 'AGACT', 'AGGAC',
       'ATAAC', 'ATGAC', 'CAAAC', 'CAGAC', 'CGAAC', 'CGGAC', 'CTAAC', 'CTGAC',
       'GAAAC', 'GAACA', 'GAACC', 'GAACT', 'GACAA', 'GACAC', 'GACAG', 'GACAT',
       'GACCA', 'GACCC', 'GACCG', 'GACCT', 'GACTA', 'GACTC', 'GACTG', 'GACTT',
       'GAGAC', 'GGAAC', 'GGACA', 'GGACC', 'GGACT', 'GGGAC', 'GTAAC', 'GTGAC',
       'TAAAC', 'TAACA', 'TAACC', 'TAACT', 'TAGAC', 'TGAAC', 'TGACA', 'TGACC',
       'TGACT', 'TGGAC', 'TTAAC', 'TTGAC', 'Relative Position', 'Count_A',
       'Count_T', 'Count_C', 'Count_G'],
      dtype='object')

### Feature selection using RandomForest's feature_importances_

In [13]:
rf = RandomForestClassifier(random_state=4262)
rf.fit(training_X.iloc[:, 2:], training_y) # Don't take into consideration the transcript name and gene name columns for training_X
rf.feature_importances_

array([3.06148276e-01, 3.50571506e-01, 2.42653612e-01, 2.43063719e-04,
       2.34765734e-04, 5.66326374e-04, 1.63151586e-05, 1.29029018e-06,
       1.17025026e-05, 3.78150658e-06, 5.55594412e-06, 9.23045171e-07,
       1.13591679e-05, 4.24428534e-06, 5.10498417e-06, 1.19083100e-06,
       4.74207514e-05, 1.70446141e-05, 1.41694717e-04, 1.37254263e-04,
       4.61534084e-04, 1.86003647e-04, 3.71825905e-03, 4.16528303e-04,
       1.10880231e-05, 1.15359341e-04, 4.39673526e-06, 1.71345760e-05,
       3.31028767e-06, 1.11940332e-04, 7.91631207e-06, 6.91854816e-05,
       3.05388051e-05, 2.79900702e-04, 3.04531830e-04, 8.03823495e-03,
       4.36163171e-05, 1.10369821e-05, 2.49524990e-04, 2.74360318e-05,
       3.57149706e-05, 1.95057836e-05, 7.07046687e-05, 1.48439371e-05,
       3.93787500e-05, 3.92903204e-06, 5.26002409e-04, 1.10391554e-04,
       2.28943482e-04, 1.42012561e-04, 4.60758204e-03, 3.40393189e-03,
       3.64823643e-02, 5.34698264e-04, 3.31339886e-05, 2.57113877e-04,
      

In [14]:
# Calculate the ranks of scores for feature importance
def calculate_rank(vector):
    a={}
    rank=1
    for num in sorted(vector, reverse=True):
        if num not in a:
            a[num]=rank
            rank=rank+1
    return[a[i] for i in vector]
feature_importance_list = sorted(list(zip(training_X.columns, rf.feature_importances_, calculate_rank(rf.feature_importances_))), key=lambda x:x[1], reverse=True)
feature_importance_list

[('Gene Name', 0.35057150587313574, 1),
 ('Transcript Name', 0.3061482760401802, 2),
 ('Mean SD', 0.24265361199536595, 3),
 ('GGACA', 0.03648236431820294, 4),
 ('TTAAC', 0.013135568837647025, 5),
 ('GAACA', 0.008038234953844306, 6),
 ('Count_T', 0.00665732219574016, 7),
 ('TTGAC', 0.005484683610792913, 8),
 ('Relative Position', 0.005050771884462054, 9),
 ('GAGAC', 0.004607582038613322, 10),
 ('AGACA', 0.0037182590517283076, 11),
 ('GGAAC', 0.003403931890689674, 12),
 ('TGACA', 0.0023803089563887597, 13),
 ('Count_A', 0.0022555314913392725, 14),
 ('TGACC', 0.0017164913947893545, 15),
 ('TAGAC', 0.0007050386408257417, 16),
 ('AAACA', 0.0005663263740365833, 17),
 ('GGACC', 0.0005346982642136358, 18),
 ('GACTA', 0.0005260024091825916, 19),
 ('TGAAC', 0.0004810454168868106, 20),
 ('AAGAC', 0.0004615340842487692, 21),
 ('AGACC', 0.00041652830296808643, 22),
 ('GAAAC', 0.00030453183012407807, 23),
 ('CTGAC', 0.00027990070241195003, 24),
 ('GGGAC', 0.0002571138766743556, 25),
 ('GACAA', 0.000

### Change the number of features to be used by the model

Tweak the argument within the `range()` function

In [15]:
top_features = []
for i in range(38): 
    top_features.append(feature_importance_list[i][0])
top_features

['Gene Name',
 'Transcript Name',
 'Mean SD',
 'GGACA',
 'TTAAC',
 'GAACA',
 'Count_T',
 'TTGAC',
 'Relative Position',
 'GAGAC',
 'AGACA',
 'GGAAC',
 'TGACA',
 'Count_A',
 'TGACC',
 'TAGAC',
 'AAACA',
 'GGACC',
 'GACTA',
 'TGAAC',
 'AAGAC',
 'AGACC',
 'GAAAC',
 'CTGAC',
 'GGGAC',
 'GACAA',
 'Mean_Mean',
 'Mean Dwelling Time',
 'GACTG',
 'TAACA',
 'AGAAC',
 'GACTT',
 'TAACT',
 'AACTG',
 'AACTT',
 'TAACC',
 'AGGAC',
 'CAGAC']

In [16]:
training_X = training_X[top_features]
validation_X = validation_X[top_features]

In [17]:
training_X

,Gene Name,Transcript Name,Mean SD,GGACA,TTAAC,GAACA,Count_T,TTGAC,Relative Position,GAGAC,...,GACTG,TAACA,AGAAC,GACTT,TAACT,AACTG,AACTT,TAACC,AGGAC,CAGAC
0,ENSG00000006451,ENST00000005257,4.900645,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,ENSG00000006451,ENST00000005257,9.120968,0,0,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000006451,ENST00000005257,2.513839,0,0,0,1,0,3,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000006451,ENST00000005257,6.078824,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,ENSG00000006451,ENST00000005257,4.955000,0,0,0,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,ENSG00000284707,ENST00000641784,2.998125,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
77117,ENSG00000284707,ENST00000641784,2.203750,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
77118,ENSG00000284707,ENST00000641784,1.874516,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
77119,ENSG00000284707,ENST00000641784,2.194032,0,0,0,2,0,2,0,...,0,0,0,0,1,0,0,0,0,0


### SMOTE

In [18]:
training_X = training_X.drop(['Transcript Name', 'Gene Name'], axis=1)
validation_X = validation_X.drop(['Transcript Name', 'Gene Name'], axis=1)

In [19]:
training_X

,Mean SD,GGACA,TTAAC,GAACA,Count_T,TTGAC,Relative Position,GAGAC,AGACA,GGAAC,...,GACTG,TAACA,AGAAC,GACTT,TAACT,AACTG,AACTT,TAACC,AGGAC,CAGAC
0,4.900645,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.120968,0,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.513839,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.078824,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4.955000,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,2.998125,0,0,1,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77117,2.203750,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77118,1.874516,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77119,2.194032,0,0,0,2,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
validation_X

,Mean SD,GGACA,TTAAC,GAACA,Count_T,TTGAC,Relative Position,GAGAC,AGACA,GGAAC,...,GACTG,TAACA,AGAAC,GACTT,TAACT,AACTG,AACTT,TAACC,AGGAC,CAGAC
0,4.764186,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.278605,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.570698,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.195217,0,0,0,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.813261,0,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33910,3.808538,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33911,2.351154,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
33912,1.922843,0,1,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33913,2.673543,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [21]:
validation_y

0        0
1        0
2        0
3        0
4        0
        ..
33910    0
33911    0
33912    0
33913    0
33914    0
Name: Label, Length: 33915, dtype: int64

In [22]:
# # Use RandomUnderSampler to sample majority class until minority class makes up 5% of the majority class counts
over = SMOTE(sampling_strategy=0.3)
training_X, training_y = over.fit_resample(training_X, training_y)
training_X

,Mean SD,GGACA,TTAAC,GAACA,Count_T,TTGAC,Relative Position,GAGAC,AGACA,GGAAC,...,GACTG,TAACA,AGAAC,GACTT,TAACT,AACTG,AACTT,TAACC,AGGAC,CAGAC
0,4.900645,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.120968,0,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.513839,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6.078824,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4.955000,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98824,5.137255,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98825,8.595984,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98826,6.062164,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98827,6.138980,0,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Model training for Non-DRACH + DRACH sites

In [ ]:
random.seed(4262)
clf = GradientBoostingClassifier(random_state=4262, max_depth=6, max_features = "log2", criterion = "friedman_mse", n_estimators = 500, loss = "log_loss", learning_rate = 0.1)
clf.fit(training_X, training_y)
y_score = clf.predict_proba(validation_X)
y_pred = clf.predict(validation_X)

In [ ]:
y_score = pd.DataFrame(y_score[:,1], columns = ['Predicted Score'])
y_pred = pd.DataFrame(y_pred, columns=['Predicted'])
validation_y = validation_y.to_frame()
y_score

In [ ]:
validation_y

In [ ]:
# Make sure that only DRACH sites are present before checking scores
y_pred_and_validation_y = pd.concat([y_score, validation_y, y_pred],axis=1)
indicator = pd.DataFrame([0,1,0] * int(len(y_pred_and_validation_y) / 3), columns = ['Indicator'])
y_pred_and_validation_y = pd.concat([y_pred_and_validation_y, indicator], axis=1)
y_pred_and_validation_y = y_pred_and_validation_y[y_pred_and_validation_y['Indicator'] == 1]
y_pred_and_validation_y = y_pred_and_validation_y.drop(['Indicator'], axis=1)
y_pred_and_validation_y

In [ ]:
# Convert DataFrame to Series before checking ROC AUC and PR AUC Score
validation_y = y_pred_and_validation_y['Label'].squeeze()
y_score = y_pred_and_validation_y['Predicted Score'].squeeze()
y_pred = y_pred_and_validation_y['Predicted'].squeeze()

In [ ]:
print('ROC_AUC score is', roc_auc_score(validation_y, y_score, average=None))

In [ ]:
# Data to plot precision - recall curve
precision, recall, thresholds = precision_recall_curve(validation_y, y_score)
# Use AUC function to calculate the area under the curve of precision recall curve
auc_precision_recall = auc(recall, precision)
print('PR AUC score is', auc_precision_recall)

In [ ]:
from sklearn.metrics import average_precision_score
average_precision_score(validation_y, y_score)

In [ ]:
from sklearn.metrics import precision_score, recall_score
print("Precision Score is", precision_score(validation_y, y_pred))
print("Recall Score is", recall_score(validation_y, y_pred))

In [ ]:
import pickle
pickle.dump(clf, open('gradientboostingclassifier.sav', 'wb'))